# Setup

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# df = pd.concat([pd.read_pickle("./uti/utiTrain.pkl"), pd.read_pickle("./utiTest.pkl")])
# use just training (screening) data
df = pd.read_pickle("./copper/copperTrain.pkl")
# Shuffle df
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,code,abstract,scibert
0,0,Background: Antioxidant micronutrients play an...,"[-0.16385548, -0.025745746, -0.1795977, 0.7668..."
1,0,Transmission of bacteria from inanimate surfac...,"[0.6531327, -0.10191646, 0.16014044, 0.7698649..."
2,0,LEARNING OBJECTIVE 1: Diagnose drug induced li...,"[-0.052812707, -0.36207545, -0.110467605, 0.48..."
3,0,BACKGROUND: Copper-oxide-impregnated linens an...,"[0.41096795, -0.20311975, -0.15459518, 0.83496..."
4,0,Pregnant women in developing countries are vul...,"[-0.033750996, -0.22547287, 0.04848912, 0.9845..."


# Helper Functions

In [3]:
# Function to plot confusion matrix
def plotConfMatrix(actual, pred, labels):
    conf_matrix = confusion_matrix(actual, pred)
    labels = ["Exclude", "Include"]
    df_cm = pd.DataFrame(conf_matrix, columns=labels, index=labels)
    df_cm.index.name = 'Actual'
    df_cm.columns.name = 'Predicted'
    plt.figure(figsize = (4,2))
    sns.set(font_scale=1.2)#for label size
    sns.heatmap(df_cm, fmt='d', cmap="Blues", annot=True, annot_kws={"size": 12})# font size
    plt.show()
    return
 
# Function to calculate probabilities of each remaining article
def calcProb(model, initial, remaining):
    # Get initial training data and labels 
    initial_data = initial['scibert'].tolist()
    initial_labels = initial['code'].tolist()

    # Fit model to initial training data
    model.fit(initial_data, initial_labels)

    # Get remaining data for testing
    remaining_data = remaining['scibert'].tolist()

    # Predict probability [exclusion, inclusion] on remaining articles
    pred = model.predict_proba(remaining_data)

    # Calculate score (positive = exclude, negative = include)
    pred = list(map(lambda x: x[0]-x[1], pred))
    # Add probability to dataframe
    remaining['prob'] = pred
    # Sort by probability
    remaining = remaining.sort_values(by=['prob']).reset_index(drop=True)
    
    return remaining

# Calculate the total number of articles screened so far (train / train + test)
def calcPercentScreened(initial, remaining):
    return (initial.index[-1] + 1) / ((remaining.index[-1] + 1) + (initial.index[-1] + 1))

# Calculate the number of articles needed to screen out of entire dataset to find all included articles
# (train + last_index / train + test)
def calcPercentNeedToScreen(initial, remaining, last_index):
    return ((initial.index[-1] + 1) + (last_index + 1)) / ((initial.index[-1] + 1) + (remaining.index[-1] + 1))

# Print stats (return false when all included articles found)
def printStats(initial, remaining):
    # Find index of last included article
    include_indicies = remaining[remaining.code == 1].index
    # Calculate total percentage of articles that need to be screened
    if (len(include_indicies) == 0):
        print("All included articles found after screening: %.2f" % (calcPercentScreened(initial, remaining) * 100))
        return False
    else:
        print("Screened: %.2f Need to screen: %.2f" % (calcPercentScreened(initial, remaining) * 100, calcPercentNeedToScreen(initial, remaining, include_indicies[-1]) * 100))
        return True

# Iterative Predition using Probabilities

In [4]:
model = LogisticRegression(C=0.05, class_weight='balanced', max_iter=1000)
all_data = df['scibert'].tolist()
all_labels = df['code'].tolist()

# Find first included article index
start = df[df.code == 1].index[0] + 1
print("Initial number screened:", start)

# Split df by minimum size to screen
initial = df.iloc[:start,:].reset_index(drop=True)
remaining = df.iloc[start:,:].reset_index(drop=True)

while(remaining.index[-1] > 0):
    remaining = calcProb(model, initial, remaining)
    if(not printStats(initial, remaining)):
        break
    # Take first remaining and append to initial (to account for extra screened article)
    initial = initial.append(remaining.iloc[0], ignore_index=True)
    remaining.drop(0, inplace=True)

Initial number screened: 45
Screened: 12.93 Need to screen: 99.71
Screened: 13.22 Need to screen: 99.71
Screened: 13.51 Need to screen: 99.71
Screened: 13.79 Need to screen: 99.71
Screened: 14.08 Need to screen: 99.71
Screened: 14.37 Need to screen: 99.71
Screened: 14.66 Need to screen: 87.64
Screened: 14.94 Need to screen: 86.21
Screened: 15.23 Need to screen: 79.60
Screened: 15.52 Need to screen: 77.01
Screened: 15.80 Need to screen: 40.23
Screened: 16.09 Need to screen: 43.10
Screened: 16.38 Need to screen: 44.25
Screened: 16.67 Need to screen: 41.38
Screened: 16.95 Need to screen: 28.16
Screened: 17.24 Need to screen: 27.87
Screened: 17.53 Need to screen: 29.89
Screened: 17.82 Need to screen: 25.00
Screened: 18.10 Need to screen: 26.44
Screened: 18.39 Need to screen: 26.44
Screened: 18.68 Need to screen: 30.75
Screened: 18.97 Need to screen: 29.60
Screened: 19.25 Need to screen: 27.59
Screened: 19.54 Need to screen: 30.46
Screened: 19.83 Need to screen: 30.46
Screened: 20.11 Need t